In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import re
import sys
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 250

In [2]:
sys.path.append('../')
import read_player_stats
import training_data
import game_by_game

In [3]:
positions = ['qb', 'rb', 'wr', 'te']
df_list = [pd.read_csv('../game_stats/'+ pos +'_game_stats.csv') for pos in positions]
df_list = [df.drop(['Unnamed: 0'], axis = 1) for df in df_list]

In [4]:
df_list[0].head()

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season,Week
0,AJFeeley,MIA,1,21,31,168,1,1,0,0,0,8.7,8.7,2004,1
1,AJFeeley,MIA,1,21,39,218,1,2,1,3,0,9.0,9.0,2004,2
2,AJFeeley,MIA,1,13,27,137,0,2,4,1,0,0.6,0.6,2004,3
3,AJFeeley,MIA,1,1,2,6,0,0,0,0,0,0.2,0.2,2004,5
4,AJFeeley,MIA,1,6,15,129,0,1,0,0,0,3.2,3.2,2004,9


In [5]:
tdata_list = [game_by_game.gbg_train_stats(df) for df in df_list]

../game_by_game.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data_frame['WeekID'] = 17*(data_frame['Season'] - orig['Season']) + (data_frame['Week'] - orig['Week'])


In [6]:
tdata_list[0].head(10)

,Label,PassComp_0,PassAtt_0,PassYards_0,PassTD_0,INT_0,RunAtt_0,RunYards_0,RunTD_0,FFP_0,WeekID_0,PassComp_1,PassAtt_1,PassYards_1,PassTD_1,INT_1,RunAtt_1,RunYards_1,RunTD_1,FFP_1,WeekID_1,PassComp_2,PassAtt_2,PassYards_2,PassTD_2,INT_2,RunAtt_2,RunYards_2,RunTD_2,FFP_2,WeekID_2,PassComp_3,PassAtt_3,PassYards_3,PassTD_3,INT_3,RunAtt_3,RunYards_3,RunTD_3,FFP_3,WeekID_3,PassComp_4,PassAtt_4,PassYards_4,PassTD_4,INT_4,RunAtt_4,RunYards_4,RunTD_4,FFP_4,WeekID_4,PassComp_5,PassAtt_5,PassYards_5,PassTD_5,INT_5,RunAtt_5,RunYards_5,RunTD_5,FFP_5,WeekID_5,PassComp_6,PassAtt_6,PassYards_6,PassTD_6,INT_6,RunAtt_6,RunYards_6,RunTD_6,FFP_6,WeekID_6,PassComp_7,PassAtt_7,PassYards_7,PassTD_7,INT_7,RunAtt_7,RunYards_7,RunTD_7,FFP_7,WeekID_7,PassComp_8,PassAtt_8,PassYards_8,PassTD_8,INT_8,RunAtt_8,RunYards_8,RunTD_8,FFP_8,WeekID_8,PassComp_9,PassAtt_9,PassYards_9,PassTD_9,INT_9,RunAtt_9,RunYards_9,RunTD_9,FFP_9,WeekID_9,PassComp_10,PassAtt_10,PassYards_10,PassTD_10,INT_10,RunAtt_10,RunYards_10,RunTD_10,FFP_10,WeekID_10,PassComp_11,PassAtt_11,PassYards_11,PassTD_11,INT_11,RunAtt_11,RunYards_11,RunTD_11,FFP_11,WeekID_11,PassComp_12,PassAtt_12,PassYards_12,PassTD_12,INT_12,RunAtt_12,RunYards_12,RunTD_12,FFP_12,WeekID_12,PassComp_13,PassAtt_13,PassYards_13,PassTD_13,INT_13,RunAtt_13,RunYards_13,RunTD_13,FFP_13,WeekID_13,PassComp_14,PassAtt_14,PassYards_14,PassTD_14,INT_14,RunAtt_14,RunYards_14,RunTD_14,FFP_14,WeekID_14,PassComp_15,PassAtt_15,PassYards_15,PassTD_15,INT_15,RunAtt_15,RunYards_15,RunTD_15,FFP_15,WeekID_15,PassComp_16,PassAtt_16,PassYards_16,PassTD_16,INT_16,RunAtt_16,RunYards_16,RunTD_16,FFP_16,WeekID_16,PassComp_17,PassAtt_17,PassYards_17,PassTD_17,INT_17,RunAtt_17,RunYards_17,RunTD_17,FFP_17,WeekID_17,PassComp_18,PassAtt_18,PassYards_18,PassTD_18,INT_18,RunAtt_18,RunYards_18,RunTD_18,FFP_18,WeekID_18,PassComp_19,PassAtt_19,PassYards_19,PassTD_19,INT_19,RunAtt_19,RunYards_19,RunTD_19,FFP_19,WeekID_19,Name
0,-1.0,12,22,156,0,1,0,0,0,3.2,131,0,0,0,0,0,0,0,0,0.0,128,20,37,175,1,0,2,5,0,10.5,126,20,33,196,0,1,1,-1,0,5.7,125,1,5,21,0,0,0,0,0,0.8,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,6.2,63,27,42,345,3,3,1,2,0,20.0,62,13,19,116,1,1,2,7,0,7.3,61,22,33,321,3,0,1,3,0,25.1,50,4,5,21,0,0,0,0,0,-0.2,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,10.0,15,22,35,198,1,0,2,-2,0,11.7,14,17,35,170,1,1,2,2,0,9.0,13,25,51,303,3,5,0,0,0,14.1,12,17,33,159,2,1,4,2,0,11.6,11,23,45,229,1,2,1,7,1,13.9,10,6,15,129,0,1,0,0,0,3.2,8,AJFeeley
1,3.2,0,0,0,0,0,0,0,0,0.0,128,20,37,175,1,0,2,5,0,10.5,126,20,33,196,0,1,1,-1,0,5.7,125,1,5,21,0,0,0,0,0,0.8,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,6.2,63,27,42,345,3,3,1,2,0,20.0,62,13,19,116,1,1,2,7,0,7.3,61,22,33,321,3,0,1,3,0,25.1,50,4,5,21,0,0,0,0,0,-0.2,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,10.0,15,22,35,198,1,0,2,-2,0,11.7,14,17,35,170,1,1,2,2,0,9.0,13,25,51,303,3,5,0,0,0,14.1,12,17,33,159,2,1,4,2,0,11.6,11,23,45,229,1,2,1,7,1,13.9,10,6,15,129,0,1,0,0,0,3.2,8,1,2,6,0,0,0,0,0,0.2,4,AJFeeley
2,0.0,20,37,175,1,0,2,5,0,10.5,126,20,33,196,0,1,1,-1,0,5.7,125,1,5,21,0,0,0,0,0,0.8,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,6.2,63,27,42,345,3,3,1,2,0,20.0,62,13,19,116,1,1,2,7,0,7.3,61,22,33,321,3,0,1,3,0,25.1,50,4,5,21,0,0,0,0,0,-0.2,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,10.0,15,22,35,198,1,0,2,-2,0,11.7,14,17,35,170,1,1,2,2,0,9.0,13,25,51,303,3,5,0,0,0,14.1,12,17,33,159,2,1,4,2,0,11.6,11,23,45,229,1,2,1,7,1,13.9,10,6,15,129,0,1,0,0,0,3.2,8,1,2,6,0,0,0,0,0,0.2,4,13,27,137,0,2,4,1,0,0.6,2,AJFeeley
3,10.5,20,33,196,0,1,1,-1,0,5.7,125,1,5,21,0,0,0,0,0,0.8,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,6.2,63,27,42,345,3,3,1,2,0,20.0,62,13,19,116,1,1,2,7,0,7.3,61,22,33,321,3,0,1,3,0,25.1,50,4,5,21,0,0,0,0,0,-0.2,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,10.0,15,22,35,198,1,0,2,-2,0,11.7,14,17,35,170,1,1,2,2,0,9.0,13,25,51,303,3,5,0,0,0,14.1,12,17,33,159,2,1,4,2,0,11.6,11,23,45,229,1,2,1,7,1,13.9,10

In [7]:
orig = {'Season': 2004, 'Week': 1}
curr_week = 17*(2015 - orig['Season']) + (1 - orig['Week'])
test_samples = [df[(df.Label == -1) & (df.WeekID_0 > curr_week-5)] for df in tdata_list]
train_samples = [df[df.Label != -1] for df in tdata_list]

In [8]:
week_labels = ['WeekID_%d' % i for i in range(20)]

In [9]:
test_samples = [df.drop(week_labels, axis=1) for df in test_samples]
train_samples = [df.drop(week_labels + ['Name'], axis=1) for df in train_samples]

In [10]:
y_train = [df['Label'] for df in train_samples]
X_train = [np.array(df.drop(['Label'], axis=1)) for df in train_samples]

In [11]:
rfr = [RandomForestRegressor(n_estimators=200) for i in range(4)]
#svr = [training_data.train_gbg_svm_model(X,y,verbose=True) for (X,y) in zip(X_train,y_train)]

In [12]:
for i in range(4):
    rfr[i].fit(X_train[i], y_train[i])

In [13]:
test_samples[0].drop(['Label', 'Name'], axis=1)

,PassComp_0,PassAtt_0,PassYards_0,PassTD_0,INT_0,RunAtt_0,RunYards_0,RunTD_0,FFP_0,PassComp_1,PassAtt_1,PassYards_1,PassTD_1,INT_1,RunAtt_1,RunYards_1,RunTD_1,FFP_1,PassComp_2,PassAtt_2,PassYards_2,PassTD_2,INT_2,RunAtt_2,RunYards_2,RunTD_2,FFP_2,PassComp_3,PassAtt_3,PassYards_3,PassTD_3,INT_3,RunAtt_3,RunYards_3,RunTD_3,FFP_3,PassComp_4,PassAtt_4,PassYards_4,PassTD_4,INT_4,RunAtt_4,RunYards_4,RunTD_4,FFP_4,PassComp_5,PassAtt_5,PassYards_5,PassTD_5,INT_5,RunAtt_5,RunYards_5,RunTD_5,FFP_5,PassComp_6,PassAtt_6,PassYards_6,PassTD_6,INT_6,RunAtt_6,RunYards_6,RunTD_6,FFP_6,PassComp_7,PassAtt_7,PassYards_7,PassTD_7,INT_7,RunAtt_7,RunYards_7,RunTD_7,FFP_7,PassComp_8,PassAtt_8,PassYards_8,PassTD_8,INT_8,RunAtt_8,RunYards_8,RunTD_8,FFP_8,PassComp_9,PassAtt_9,PassYards_9,PassTD_9,INT_9,RunAtt_9,RunYards_9,RunTD_9,FFP_9,PassComp_10,PassAtt_10,PassYards_10,PassTD_10,INT_10,RunAtt_10,RunYards_10,RunTD_10,FFP_10,PassComp_11,PassAtt_11,PassYards_11,PassTD_11,INT_11,RunAtt_11,RunYards_11,RunTD_11,FFP_11,PassComp_12,PassAtt_12,PassYards_12,PassTD_12,INT_12,RunAtt_12,RunYards_12,RunTD_12,FFP_12,PassComp_13,PassAtt_13,PassYards_13,PassTD_13,INT_13,RunAtt_13,RunYards_13,RunTD_13,FFP_13,PassComp_14,PassAtt_14,PassYards_14,PassTD_14,INT_14,RunAtt_14,RunYards_14,RunTD_14,FFP_14,PassComp_15,PassAtt_15,PassYards_15,PassTD_15,INT_15,RunAtt_15,RunYards_15,RunTD_15,FFP_15,PassComp_16,PassAtt_16,PassYards_16,PassTD_16,INT_16,RunAtt_16,RunYards_16,RunTD_16,FFP_16,PassComp_17,PassAtt_17,PassYards_17,PassTD_17,INT_17,RunAtt_17,RunYards_17,RunTD_17,FFP_17,PassComp_18,PassAtt_18,PassYards_18,PassTD_18,INT_18,RunAtt_18,RunYards_18,RunTD_18,FFP_18,PassComp_19,PassAtt_19,PassYards_19,PassTD_19,INT_19,RunAtt_19,RunYards_19,RunTD_19,FFP_19
0,18,23,189,3,0,8,35,0,23.1,17,22,226,2,0,2,14,1,24.4,31,40,318,1,0,0,0,0,15.7,17,42,185,0,2,3,27,0,6.1,24,36,327,3,0,5,28,0,27.9,24,38,368,2,0,5,22,0,24.9,19,29,209,2,0,6,34,0,19.8,22,36,341,3,0,3,32,0,28.8,18,27,315,6,0,0,0,0,36.6,28,39,418,1,2,2,21,1,24.8,19,22,255,3,0,3,21,0,24.3,24,42,264,3,0,7,34,0,26.0,12,17,156,3,0,0,0,0,18.2,22,28,302,4,0,1,8,0,28.9,16,27,162,1,0,0,0,0,10.5,25,42,346,3,0,6,28,0,27.6,23,33,189,1,1,0,0,0,9.6,25,39,318,2,2,1,5,0,17.2,1,2,27,0,0,1,9,0,2.0,24,29,285,2,0,6,31,0,22.5
0,22,33,243,3,0,9,15,0,23.2,31,45,311,0,0,2,14,0,13.8,18,30,297,2,0,3,17,0,21.6,26,39,293,1,1,4,26,0,16.3,15,23,153,2,1,5,6,0,12.7,20,36,234,2,0,2,5,0,17.9,11,16,108,0,0,2,-1,0,4.2,17,29,177,0,0,4,25,1,15.6,21,31,199,2,0,0,0,0,16.0,24,28,226,0,0,4,24,0,10.4,19,28,221,1,0,6,29,0,15.7,16,30,158,1,1,1,6,0,8.9,20,26,248,3,0,4,8,0,22.7,19,25,186,3,0,2,17,0,20.1,26,42,255,0,0,5,42,0,14.4,19,35,202,1,3,6,36,0,9.7,16,29,153,0,2,6,47,0,5.8,17,20,287,5,0,4,17,0,32.2,14,20,137,2,0,2,7,0,14.2,26,42,293,2,1,4,46,0,22.3
0,26,49,243,2,2,4,20,0,15.7,10,16,160,2,0,1,5,0,14.9,15,22,109,0,2,0,0,0,0.4,18,34,187,2,1,8,18,0,15.3,24,53,294,2,2,5,37,1,24.5,19,27,370,5,1,4,20,0,33.8,21,32,253,1,0,8,49,0,17.0,23,39,303,2,1,3,15,0,19.6,25,46,354,4,0,2,5,0,30.7,26,45,400,3,2,3,26,0,26.6,27,42,344,2,0,4,5,0,21.3,25,44,370,3,1,7,18,0,25.6,32,49,312,1,2,6,12,1,19.7,29,41,393,4,1,1,-1,0,29.6,31,39,370,4,0,3,15,0,32.3,20,34,172,3,1,4,20,0,18.9,35,53,370,2,2,5,19,1,26.7,26,37,282,1,0,2,7,0,16.0,26,37,241,1,0,6,5,0,14.1,19,32,180,2,1,5,29,0,16.1
0,25,34,269,2,0,4,1,0,18.9,27,38,244,2,2,3,8,0,14.6,17,26,146,2,1,6,25,0,14.3,14,24,117,0,1,2,3,0,3.0,21,29,302,2,0,4,22,1,27.3,19,27,176,1,3,3,7,1,11.7,24,35,233,1,1,7,6,0,11.9,16,22,220,3,0,5,12,0,22.0,10,33,86,0,3,3,8,0,-1.8,19,31,233,2,2,2,11,0,14.4,21,28,266,0,1,6,14,2,21.0,18,38,126,0,0,1,0,0,5.0,34,44,323,2,2,4,25,0,19.4,15,24,204,2,0,2,16,0,17.8,15,23,169,0,1,3,3,0,5.1,15,23,252,1,0,3,6,0,14.7,25,38,301,1,0,6,3,0,16.3,21,36,281,2,4,6,23,1,19.5,27,38,366,4,0,1,10,0,31.6,25,44,230,2,0,4,20,0,19.2
0,26,38,351,1,1,0,0,0,16.0,24,38,317,2,1,4,3,0,18.0,18,25,220,1,0,4,-4,0,12.4,27,35,360,0,0,2,-2,0,14.2,25,39,350,3,0,1,-1,0,25.9,32,58,435,2,2,1,8,0,22.2,21,32,207,1,1,3,2,0,10.5,30,43,343,1,2,1,0,0,13.7,25,37,340,6,0,2,1,0,3

In [14]:
preds = [rfr[i].predict(np.array(test_samples[i].drop(['Label', 'Name'], axis=1))) for i in range(4)]

In [15]:
proj_list = []
for i in range(4):
    test_samples[i]['Projection'] = preds[i]
    pro_df = test_samples[i][['Name', 'Projection']]
    proj_list.append(pro_df.sort(columns=['Projection'], ascending=False))

In [16]:
proj_list[0]

,Name,Projection
0,AaronRodgers,19.2645
0,AndrewLuck,17.9690
0,MattRyan,17.6955
0,NickFoles,17.6690
0,BenRoethlisberger,17.6260
0,DrewBrees,17.1085
0,RussellWilson,16.8855
0,RyanTannehill,16.6515
0,PeytonManning,16.4410
0,EliManning,16.0610


In [17]:
proj_list[1]

,Name,Projection
0,LeVeonBell,17.4970
0,MattForte,17.0615
0,DeMarcoMurray,15.9080
0,MarshawnLynch,15.4345
0,JamaalCharles,14.9500
0,EddieLacy,14.3310
0,ArianFoster,13.9110
0,AdrianPeterson,12.6510
0,MarkIngram,11.8690
0,AndreEllington,11.6165


In [18]:
proj_list[2].drop(['JordyNelson', 'JoshGordon'], inplace=True)
proj_list[2]

,Name,Projection
0,DemaryiusThomas,19.1255
0,JordyNelson,16.6675
0,AntonioBrown,15.6965
0,JoshGordon,15.4175
0,JulioJones,15.3605
0,JulianEdelman,14.7580
0,TYHilton,14.6510
0,CalvinJohnson,14.5550
0,DezBryant,14.5350
0,AJGreen,14.0370


In [19]:
proj_list[3]

,Name,Projection
0,RobGronkowski,12.5155
0,JimmyGraham,11.4325
0,MartellusBennett,11.1405
0,GregOlsen,10.0545
0,CobyFleener,9.8120
0,AntonioGates,9.7495
0,DelanieWalker,9.2765
0,JordanCameron,9.2425
0,ZachErtz,9.1975
0,JuliusThomas,8.8240


In [20]:
positions = ['QB', 'RB', 'WR', 'TE']
pos_salaries = []
for pos in positions:
    pos_salaries.append(pd.read_csv('../salaries/by_position/' + pos + '_W2_2015.csv'))

In [21]:
week_table = []
for proj, sal in zip(proj_list, pos_salaries):
    week_table.append(pd.merge(proj, sal, on='Name'))

In [22]:
for table in week_table:
    table['DPP'] = table['Salary']/table['Projection']

In [23]:
week_table[0]

,Name,Projection,Unnamed: 0,Salary,Position,DPP
0,AaronRodgers,19.2645,0,9300,QB,482.753251
1,MattRyan,17.6955,6,8700,QB,491.650420
2,NickFoles,17.6690,74,6500,QB,367.875941
3,BenRoethlisberger,17.6260,11,8400,QB,476.568705
4,DrewBrees,17.1085,3,8900,QB,520.209253
5,RussellWilson,16.8855,8,8500,QB,503.390483
6,RyanTannehill,16.6515,28,8000,QB,480.437198
7,EliManning,16.0610,30,8000,QB,498.100990
8,TonyRomo,15.9875,9,8500,QB,531.665364
9,TomBrady,15.8255,14,8300,QB,524.470001


In [24]:
week_table[1]

,Name,Projection,Unnamed: 0,Salary,Position,DPP
0,MattForte,17.0615,4,8900,RB,521.642294
1,DeMarcoMurray,15.9080,13,8300,RB,521.750063
2,MarshawnLynch,15.4345,10,8500,RB,550.714309
3,EddieLacy,14.3310,25,8100,RB,565.208290
4,ArianFoster,13.9110,17,8200,RB,589.461577
5,AdrianPeterson,12.6510,2,9000,RB,711.406213
6,MarkIngram,11.8690,29,8000,RB,674.024770
7,AndreEllington,11.6165,36,7700,RB,662.850256
8,RashadJennings,11.5860,79,6400,RB,552.390817
9,GiovaniBernard,11.5130,51,7000,RB,608.008338


In [25]:
week_table[2]

,Name,Projection,Unnamed: 0,Salary,Position,DPP
0,AntonioBrown,15.6965,1,9200,WR,586.117924
1,JulioJones,15.3605,5,8800,WR,572.898018
2,JulianEdelman,14.7580,37,7600,WR,514.974929
3,CalvinJohnson,14.5550,21,8100,WR,556.509790
4,AJGreen,14.0370,23,8100,WR,577.046377
5,KeenanAllen,12.9565,18,8200,WR,632.886968
6,RandallCobb,12.7190,16,8200,WR,644.704772
7,DarriusHeywardBey,12.5805,498,4500,WR,357.696435
8,GoldenTate,12.3565,62,6700,WR,542.224740
9,DeAndreHopkins,11.9820,22,8100,WR,676.014021


In [26]:
week_table[3]

,Name,Projection,Unnamed: 0,Salary,Position,DPP
0,RobGronkowski,12.5155,15,8300,TE,663.177660
1,JimmyGraham,11.4325,50,7100,TE,621.036519
2,MartellusBennett,11.1405,95,6100,TE,547.551726
3,GregOlsen,10.0545,103,6000,TE,596.747725
4,DelanieWalker,9.2765,178,5200,TE,560.556244
5,JordanCameron,9.2425,128,5500,TE,595.077090
6,ZachErtz,9.1975,183,5200,TE,565.371025
7,JuliusThomas,8.8240,138,5400,TE,611.967362
8,MychalRivera,8.6705,235,4900,TE,565.134652
9,JasonWitten,8.6045,101,6000,TE,697.309547


In [29]:
week_table = [table.drop(['Unnamed: 0'], axis=1) for table in week_table]
positions = ['QB', 'RB', 'WR', 'TE']
for pos, table in zip(positions,week_table):
    table.to_csv('../projections/weekly/2015_w2/' + pos + '_proj_and_sal.csv')